<a href="https://colab.research.google.com/github/rhozon/Banca-FAE/blob/master/RL_CommoditiesAgricolas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reinforcement Learning Trading Strategy para Commodities Agrícolas

Este notebook adapta o estudo de caso original de Reinforcement Learning para trading, usando yahooquery para ingestão de dados e aplicando a estratégia aos tickers de commodities agrícolas.

**Análise Entregue:**

- Para cada ticker, calcula o lucro total acumulado em cada episódio de treinamento.

- Gera um vetor results[ticker] com o lucro por episódio.

- Plota a evolução do lucro total ao longo dos episódios, permitindo comparar a performance entre commodities.

In [1]:
#@title Instalação das dependências libs
!pip install yahooquery gymnasium torch numpy pandas matplotlib yfinance --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.4 MB/s eta 0:00:00


In [22]:
#@title Python lib imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from yahooquery import Ticker
import yfinance as yf
from collections import deque

import gymnasium as gym
from gymnasium import spaces

import torch
import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")


Definimos os tickers de commodities agrícolas e implementamos a função ``fetch_close_prices`` para recuperar preços de fechamento.

In [13]:
!pip install rpy2 --quiet

In [14]:
%load_ext rpy2.ipython

In [32]:
%%R

# Instale os pacotes se necessário:
install.packages(c("quantmod", "dplyr"))

suppressPackageStartupMessages({
library(quantmod)
library(dplyr)
})

fetch_close_prices_qm <- function(tickers, start, end, cache_path = "prices_qm.csv") {
  # Se já existe CSV em cache, carrega e retorna
  if (file.exists(cache_path)) {
    df <- read.csv(cache_path, stringsAsFactors = FALSE) %>%
      mutate(date = as.Date(date))
    message("Loaded from cache: ", cache_path)
    return(df)
  }

  # Senão, faz o download para cada ticker
  all_data <- lapply(tickers, function(tk) {
    # getSymbols retorna um objeto xts com colunas Open, High, Low, Close, Volume, Adjusted
    xts_data <- getSymbols(tk, src = "yahoo", from = start, to = end, auto.assign = FALSE)
    close_prices <- Ad(xts_data)  # usa Preço Ajustado (Adjusted Close)
    data.frame(
      date   = index(close_prices),
      ticker = tk,
      close  = as.numeric(close_prices),
      row.names = NULL
    )
  })

  # Combina tudo em um data.frame
  df <- bind_rows(all_data)

  # Salva em CSV para próximas execuções
  write.csv(df, cache_path, row.names = FALSE)
  message("Saved to cache: ", cache_path)

  return(df)
}

# Exemplo de uso:
tickers    <- c("ZC=F","ZO=F","KE=F","GF=F","ZS=F","ZM=F","ZL=F")
start_date <- "2020-01-01"
end_date   <- "2025-05-01"

df_prices <- fetch_close_prices_qm(tickers, start_date, end_date)
tail(df_prices)


           date ticker close
9382 2025-04-23   ZL=F 47.91
9383 2025-04-24   ZL=F 49.65
9384 2025-04-25   ZL=F 49.28
9385 2025-04-28   ZL=F 49.91
9386 2025-04-29   ZL=F 48.85
9387 2025-04-30   ZL=F 48.58


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/quantmod_0.4.27.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/dplyr_1.1.4.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpzxQp4Z/downloaded_packages’
Loaded from cache: prices_qm.csv


In [23]:

df_prices = pd.read_csv('prices_qm.csv', parse_dates=['date'])
df_prices.tail()


,date,ticker,close
9382,2025-04-24,ZL=F,49.650002
9383,2025-04-25,ZL=F,49.279999
9384,2025-04-28,ZL=F,49.910000
9385,2025-04-29,ZL=F,48.849998
9386,2025-04-30,ZL=F,48.580002


## Preparação do Estado (getState)

Implementamos a função que transforma uma janela de preços em um vetor de retornos normalizados.

In [24]:

def getState(data, t, window_size):
    """
    Converte uma janela de preços em vetor de retornos normalizados.
    """
    # Inicia índice da janela
    d = t - window_size + 1
    # Se a janela ultrapassar início, preenche com primeiro preço
    block = data[d:t+1] if d >= 0 else -d * [data[0]] + list(data[0:t+1])
    # Calcula retorno percentual entre pares consecutivos
    res = [(block[i+1] - block[i]) / block[i] for i in range(len(block)-1)]
    return np.array(res, dtype=np.float32)


## Definição do Agente de RL

Definição do Agente de RL com Q-Learning Estável

In [25]:

class Agent(nn.Module):
    def __init__(
        self,
        state_size,
        hidden_size=64,
        lr=1e-4,
        gamma=0.95,
        epsilon=1.0,
        epsilon_min=0.01,
        epsilon_decay=0.995
    ):
        super(Agent, self).__init__()
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        # Rede para estimar Q-values
        self.model = nn.Sequential(
            nn.Linear(state_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 3)  # Q para 3 ações
        )
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

    def act(self, state):
        # ε-greedy: explora ou explora
        if np.random.rand() < self.epsilon:
            return np.random.choice([0,1,2])
        state_t = torch.from_numpy(state).unsqueeze(0)
        q_values = self.model(state_t).detach().numpy()[0]
        return np.argmax(q_values)

    def train_step(self, state, action, reward, next_state):
        state_t = torch.from_numpy(state).unsqueeze(0)
        next_t = torch.from_numpy(next_state).unsqueeze(0)
        # Q atual e Q do próximo estado
        q_values = self.model(state_t)
        with torch.no_grad():
            q_next = self.model(next_t).max(1)[0]
        # Monta target: only update o Q do action realizado
        target = q_values.clone().detach()
        target[0, action] = reward + self.gamma * q_next
        # Backprop
        loss = self.criterion(q_values, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        # Decai epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


## Loop de Treinamento e Avaliação

Treinamos o agente por vários episódios para cada ticker, armazenando o lucro total de cada episódio.

In [26]:
window_size = 10
episodes    = 50
results     = {}

for tk in tickers:
    print(f"\n=== Treinando para {tk} ===")
    # Prepara série de preços para o ticker
    prices = df_prices[df_prices.ticker == tk].sort_values("date")['close'].values
    agent  = Agent(window_size)
    total_profits = []

    for e in range(episodes):
        state = getState(prices, 0, window_size+1)
        agent.inventory = []
        total_profit   = 0

        for t in range(len(prices)-1):
            action     = agent.act(state)
            next_state = getState(prices, t+1, window_size+1)
            reward     = 0

            # Executa ação: BUY, SELL ou HOLD
            if action == 1:  # BUY
                agent.inventory.append(prices[t])
            elif action == 2 and agent.inventory:  # SELL
                bought_price = agent.inventory.pop(0)
                profit       = prices[t] - bought_price
                reward       = profit / bought_price  # recompensa normalizada
                total_profit += profit

            # Atualiza rede com Q-Learning
            agent.train_step(state, action, reward, next_state)
            state = next_state

        total_profits.append(total_profit)
        if (e+1) % 10 == 0:
            print(f"Episode {e+1}/{episodes} — Lucro: {total_profit:.2f}")

    results[tk] = total_profits



=== Treinando para ZC=F ===
Episode 10/50 — Lucro: -328.50
Episode 20/50 — Lucro: 16.50
Episode 30/50 — Lucro: 227.50
Episode 40/50 — Lucro: 170.50
Episode 50/50 — Lucro: -229.75

=== Treinando para ZO=F ===
Episode 10/50 — Lucro: 121.75
Episode 20/50 — Lucro: 0.00
Episode 30/50 — Lucro: -353.50
Episode 40/50 — Lucro: -312.75
Episode 50/50 — Lucro: 112.50

=== Treinando para KE=F ===
Episode 10/50 — Lucro: -136.25
Episode 20/50 — Lucro: -230.50
Episode 30/50 — Lucro: 172.75
Episode 40/50 — Lucro: -754.50
Episode 50/50 — Lucro: 314.75

=== Treinando para GF=F ===
Episode 10/50 — Lucro: -9.43
Episode 20/50 — Lucro: 118.23
Episode 30/50 — Lucro: 151.65
Episode 40/50 — Lucro: 83.47
Episode 50/50 — Lucro: 61.03

=== Treinando para ZS=F ===
Episode 10/50 — Lucro: 909.00
Episode 20/50 — Lucro: 1.00
Episode 30/50 — Lucro: -337.25
Episode 40/50 — Lucro: 789.50
Episode 50/50 — Lucro: 348.50

=== Treinando para ZM=F ===
Episode 10/50 — Lucro: 177.30
Episode 20/50 — Lucro: 301.30
Episode 30/50 — 

## Plotagem dos Resultados

Visualizamos a evolução do lucro total por episódio para cada commodity.

In [27]:

import plotly.express as px

# Converta o dicionário de resultados em DataFrame
df_hist = pd.DataFrame(results)
df_hist['Episódio'] = df_hist.index

#  formato longo, que o plotly gosta
df_melt = df_hist.melt(
    id_vars='Episódio',
    var_name='ticker',
    value_name='Lucro'
)

# Plote com Plotly Express
fig = px.line(
    df_melt,
    x='Episódio',
    y='Lucro',
    color='ticker',
    title='Evolução do Lucro Total por Episódio'
)
fig.update_layout(
    xaxis_title='Episódio',
    yaxis_title='Lucro Total (USD)'
)
fig.show()


## Interpretação Atualizada dos Resultados

O gráfico mostra resultados moderados, sem picos extremos, indicando que o agente está **aprendendo de forma mais estável**, porém ainda com bastante ruído. Veja como entender cada padrão:

---

### 1. Episódio inicial com lucro elevado (ZC=F)

- No episódio 0, o ticker **ZC=F** obteve um lucro alto (~6000 USD) devido a uma sequência de compras e vendas que coincidiram com movimentos de preço no início do treinamento.
- Esse valor atípico provavelmente reflete **ações exploratórias** intensas com ε próximo a 1.  

---

### 2. Convergência para lucros moderados

- A partir do episódio 1 em diante, todos os tickers **flutuam ao redor de zero** com ganhos e perdas moderados (<±2000 USD).
- Isso sugere que o agente já **reduziu a exploração** (ε decaído) e começa a “assentar” sua política em padrões menos arriscados.  

---

### 3. Ruído persistente

- Apesar da convergência inicial, ainda há **oscilações**:  
  - Alguns episódios geram ganhos pontuais (ex.: ZS=F nos episódios ~5 e ~8).  
  - Outros produzem perdas ligeiras (ex.: KE=F no episódio ~25).  
- Esse **ruído** pode indicar que o agente não encontrou padrão suficientemente forte, ou que a rede está subajustada (underfitting).

---

### 4. Ausência de tendência crescente

- **Não há uma curva claramente ascendente** ao longo dos episódios.  
- Em um cenário de aprendizado ideal, o lucro médio por episódio tenderia a crescer de forma suave — aqui, isso não ocorre.

---

### 5. Próximos ajustes recomendados

1. **Diminuir ainda mais ε** após os episódios iniciais para focar na exploração das melhores ações aprendidas.  
2. **Aumentar a complexidade da rede** (camadas/neuronios) para capturar padrões mais sutis.  
3. **Implementar Replay Buffer** e **Target Network** para reduzir o ruído e estabilizar o Q-learning.  
4. **Ajustar funções de recompensa** (ex.: penalizar holding prolongado) para incentivar decisões mais decisivas.  

Com essas observações, você poderá direcionar esforços para estabilizar o aprendizado e extrair sinais de trading mais confiáveis.  


# Extração de Sinais de Compra e Venda

Para transformar a política aprendida em sinais de compra e venda, podemos usar o agente treinado (com ε reduzido) para gerar um histórico de ações em um episódio “simulado”. Exemplo para um ticker tk:

In [28]:
# 9.1 Defina epsilon para exploração mínima
agent.epsilon = agent.epsilon_min

# 9.2 Simule um episódio e registre sinais
signals = []
prices_tk = df_prices[df_prices.ticker == tk].sort_values('date')
dates = prices_tk['date'].values
values = prices_tk['close'].values
state = getState(values, 0, window_size+1)
agent.inventory = []

for t in range(len(values)-1):
    action = agent.act(state)
    date   = dates[t]
    price  = values[t]
    if action == 1:
        signals.append({'date': date, 'action': 'BUY',  'price': price})
        agent.inventory.append(price)
    elif action == 2 and agent.inventory:
        signals.append({'date': date, 'action': 'SELL', 'price': price})
        agent.inventory.pop(0)
    # avança estado
    state = getState(values, t+1, window_size+1)

signals_df = pd.DataFrame(signals)
signals_df.tail()


,date,action,price
3,2022-02-04,SELL,65.360001
4,2022-03-14,BUY,78.730003
5,2023-03-01,BUY,60.540001
6,2023-11-30,BUY,52.290001
7,2023-12-01,BUY,51.380001


In [30]:
#@title Visualização dos Sinais no Gráfico de Preço

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 1) Gera sinais para cada ticker
all_signals = {}
for tk in tickers:
    # usa epsilon mínimo
    agent.epsilon = agent.epsilon_min
    prices_tk = df_prices[df_prices.ticker==tk].sort_values('date')
    dates  = prices_tk['date'].values
    values = prices_tk['close'].values
    state = getState(values, 0, window_size+1)
    agent.inventory = []
    signals = []

    for t in range(len(values)-1):
        action = agent.act(state)
        date   = dates[t]
        price  = values[t]
        if action == 1:
            signals.append({'date': date, 'action': 'BUY',  'price': price})
            agent.inventory.append(price)
        elif action == 2 and agent.inventory:
            signals.append({'date': date, 'action': 'SELL', 'price': price})
            agent.inventory.pop(0)
        state = getState(values, t+1, window_size+1)

    all_signals[tk] = pd.DataFrame(signals)

# 2) Cria figura com uma linha por ticker
fig = make_subplots(
    rows=len(tickers), cols=1,
    shared_xaxes=True,
    subplot_titles=tickers,
    vertical_spacing=0.02
)

for i, tk in enumerate(tickers, start=1):
    prices_tk = df_prices[df_prices.ticker==tk].sort_values('date')
    sig_df    = all_signals[tk]

    # Preço
    fig.add_trace(
        go.Scatter(x=prices_tk['date'], y=prices_tk['close'], mode='lines', name=f'Preço {tk}'),
        row=i, col=1
    )
    # Buy
    fig.add_trace(
        go.Scatter(x=sig_df.query("action=='BUY'")['date'],
                   y=sig_df.query("action=='BUY'")['price'],
                   mode='markers', marker_symbol='triangle-up',
                   marker_size=8, marker_color='green', showlegend=False),
        row=i, col=1
    )
    # Sell
    fig.add_trace(
        go.Scatter(x=sig_df.query("action=='SELL'")['date'],
                   y=sig_df.query("action=='SELL'")['price'],
                   mode='markers', marker_symbol='triangle-down',
                   marker_size=8, marker_color='red', showlegend=False),
        row=i, col=1
    )

fig.update_layout(
    height=300 * len(tickers),
    title_text='Sinais RL por Ticker',
    showlegend=True
)
fig.update_yaxes(title_text="Preço (USD)")
fig.update_xaxes(title_text="Data")
fig.show()

